In [1]:
# Neural Network for binary classification and regression tasks

In [2]:
import numpy as np

In [3]:
# with sigmoid as activation (you can remove sigmoid to perform regression tasks)
# if you want to perform multi-label classification fuck you
class NeuralNetwork:
    def __init__(self, no_of_layers):
        self.num_layers = no_of_layers
        num_neurons = []
        hidden_count = 1
        for i in range(no_of_layers):
            if i != no_of_layers - 1:
                print("Enter Number of Neurons for Hidden Layer "+str(hidden_count))
                num_neurons.append(int(input()))
                hidden_count += 1
            else:
                print("Number of neurons in output layer = 1")
                num_neurons.append(1)
                
            
        self.numb_neurons = num_neurons
        self.weights = []
        self.bias = []
        self.a = []
        self.dz = []
        self.delta_w = []
        self.delta_b = []
        self.y_pred = None
        self.trained = False
        self.lr = 0
        self.m = 0
    def train(self, train_x, train_y, epochs, lr):
        # weight and bias initializations
        if self.trained == False:
            for i in range(self.num_layers):
                if i == 0:
                   self.weights.append(np.random.randn(self.numb_neurons[i],train_x.shape[1]))
                else:
                   self.weights.append(np.random.randn(self.numb_neurons[i],self.numb_neurons[i - 1]))
                self.bias.append(np.random.randn(self.numb_neurons[i],1))
                self.y_pred = np.zeros((1,len(train_y)))
                self.lr = lr
                self.m = len(train_x)
        self.trained = True
        for i in range(epochs):    
            self.forward_pass(train_x)
            self.backward_pass(train_x, train_y)
            y_t = train_y.reshape(1, len(train_y))
            print(self.loss_func(y_t, self.y_pred))

    def test(self, x_test):
        a_ = []
        
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], x_test.T) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            else:
                z = np.matmul(self.weights[i], a_[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            y_p = a[-1].flatten()
            # only for classification tasks
            for i in range(len(y_p)):
                if y_p[i] >= 0.5:
                   y_p[i] = 1
                else:
                   y_p[i] = 0
        return y_p

    
    def loss_func(self, y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) 
    
    def forward_pass(self, train_x):
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], train_x.T) + self.bias[i]
                a = 1/(1+np.exp(-z)) # remove this to perform regression tasks
                self.a.append(a)
            else:
                z = np.matmul(self.weights[i], self.a[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                self.a.append(a)

    def backward_pass(self, train_x, train_y):
        for i in range(self.num_layers - 1, -1,-1):
            # last layer case where derivative of loss is needed
            if i == self.num_layers - 1:
                dz = self.a[i] - train_y
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz ,self.a[i-1].T)  
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # first layer case where a[i-1] is actually train_x
            elif i == 0:               
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, train_x)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # all hidden layer cases
            else:
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, self.a[i-1].T)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
        # updating weights
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * self.delta_w[i]
            self.bias[i] -= self.lr * np.broadcast_to(self.delta_b[i],self.bias[i].shape)
        self.y_pred = self.a[-1]
        self.dz, self.a, self.delta_w, self.delta_b  = [],[],[],[]                

In [4]:
nn = NeuralNetwork(6)

Enter Number of Neurons for Hidden Layer 1


 100


Enter Number of Neurons for Hidden Layer 2


 64


Enter Number of Neurons for Hidden Layer 3


...

Enter Number of Neurons for Hidden Layer 4


 16


Enter Number of Neurons for Hidden Layer 5


 8


Number of neurons in output layer = 1


In [5]:
# clean and normalize your data here
train_x = np.array([[0,0],[0,1],[1,0],[1,1]])
train_y = np.array([0,0,0,1])

In [6]:
# train_x should be scaled and of shape (training examples, features) i.e (batch_size, input)
# train_y should be a one dimensional vector
# both train_x and train_y must be numpy arrays
# 3rd param is no. of epochs
# 4th param is learning rate
nn.train(train_x,train_y,1000, 0.1)

0.629957205006649
0.5691900696684941
0.5416575620629337
0.5270281536503301
0.5174179668846146
0.5099598503846691
0.5035475602115294
0.4977206403485871
0.49227161419026444
0.4870972865709403
0.48213907780398846
0.4773583026937771
0.47272586708999464
0.4682180444426051
0.4638148028859348
0.459499167316884
0.45525697095058804
0.4510767275495924
0.44694951527958726
0.4428688287579733
0.4388303826680639
0.4348318627809957
0.4308726284033838
0.4269533773716138
0.4230757908123256
0.4192421788757361
0.4154551494458416
0.41171731909684417
0.408031079918161
0.40439842863503456
0.4008208573195537
0.3972992992541981
0.3938341199084331
0.3904251415926613
0.3870716907491801
0.3837726583862069
0.3805265662244589
0.3773316332196577
0.3741858389541015
0.3710869818309884
0.3680327310488755
0.365020672036653
0.3620483454705411
0.3591132802504985
0.3562130209467017
0.3533451502833118
0.3505073072375313
0.34769720131649595
0.3449126235444778
0.3421514546549501
0.3394116709401115
0.3366913481668117
0.333988

In [7]:
nn.test(np.array([[0,0],[0,1],[1,0],[1,1]]))

array([0., 0., 0., 1.])